# 数据检索

用户输入只是问题，答案需要从已知的数据库或者文档中获取。因此，我们需要使用检索器获取上下文，并通过“question”键下的用户输入结合上下文来回答问题。

### RAG
检索增强生成（Retrieval-augmented Generation，RAG），是当下最热门的大模型前沿技术之一。如果将“微调（finetune）”理解成大模型内化吸收知识的过程，那么RAG就相当于给大模型装上了“知识外挂”，基础大模型不用再训练即可随时调用特定领域知识。

In [1]:
# 导入本地词嵌入模型
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="/home/libing/kk_LLMs/bge-large-zh-v1.5")

/tmp/ipykernel_13789/2988834985.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="/home/libing/kk_LLMs/bge-large-zh-v1.5")
/home/libing/miniconda3/envs/kk_langchain_wsl/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [2]:
# 构建上下文向量数据库
texts = [
    "今天天气很好，适合出去游玩。", 
    "今天天气不好，不适合出去游玩。",
    "小明在华为工作，是一个非常优秀的青年。",
    '熊喜欢吃蜂蜜，尤其是新鲜的蜂蜜。',
    '狗喜欢吃屎，尤其是热乎乎的新鲜的狗屎。'
]
vector_store = FAISS.from_texts(texts, embeddings)
vector_store

In [3]:
# 使用向量数据库生成检索器
retriever = vector_store.as_retriever()

In [4]:
retriever.invoke("今天天气怎么样？")

[Document(id='c26cba9c-2ceb-46f6-a9e1-64c3911e5dbb', metadata={}, page_content='今天天气很好，适合出去游玩。'),
 Document(id='d6ca30ea-e422-4c23-8ce0-06fd1a43688c', metadata={}, page_content='今天天气不好，不适合出去游玩。'),
 Document(id='ef13f868-d350-4533-bdc2-03b97ea90928', metadata={}, page_content='狗喜欢吃屎，尤其是热乎乎的新鲜的狗屎。'),
 Document(id='1768594a-596f-4df6-928f-ab955f126e1e', metadata={}, page_content='熊喜欢吃蜂蜜，尤其是新鲜的蜂蜜。')]

In [5]:
# 接入本地部署的大模型
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("LOCAL_API_KEY")
base_url = os.getenv("LOCAL_API_BASE")

llm = ChatOpenAI(api_key=api_key, base_url=base_url, temperature=0.3, max_tokens=8192)


In [6]:
# 构建提示词模板
template = """
请根据以下文档回答问题:
{context}

问题:
{question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [7]:
# 构建链
from langchain_core.runnables import RunnablePassthrough

output_parser = StrOutputParser()

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | output_parser
)

chain.invoke("今天不错，是否适合出去游玩？")

"根据提供的文档内容，有两个文档提到了天气情况：\n\n1. Document(id='c26cba9c-2ceb-46f6-a9e1-64c3911e5dbb', metadata={}, page_content='今天天气很好，适合出去游玩。')\n2. Document(id='d6ca30ea-e422-4c23-8ce0-06fd1a43688c', metadata={}, page_content='今天天气不好，不适合出去游玩。')\n\n第一个文档指出“今天天气很好，适合出去游玩”，而第二个文档则说“今天天气不好，不适合出去游玩”。由于这两个文档给出了相互矛盾的信息，我们无法确定今天的天气究竟如何。因此，无法根据这些信息明确回答是否适合出去游玩。"

In [8]:
chain.invoke("狗熊喜欢吃吃什么？")

'根据提供的文档内容，我们可以看到有两段描述了动物的饮食偏好：\n\n1. "熊喜欢吃蜂蜜，尤其是新鲜的蜂蜜。"\n2. "狗喜欢吃屎，尤其是热乎乎的新鲜的狗屎。"\n\n因此，根据这些信息，我们可以得出结论：\n- 熊喜欢吃蜂蜜。\n- 狗喜欢吃屎（这里可能是指狗吃自己的粪便，这是一种在某些情况下可能发生的行为）。\n\n由于问题中提到的“狗熊”这个词通常指的是熊类动物，而不是真正的狗和熊的组合体，所以我们可以推断出狗熊喜欢吃的食物是蜂蜜。'

In [9]:
# 在回答问题的时候加上称呼
from operator import itemgetter

new_template = """\
只根据以下上下文回答问题: 
{context}

问题: {question}
回答问题的时候请加上称呼: {name}    
"""
new_prompt = ChatPromptTemplate.from_template(new_template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "name": itemgetter("name")
    }
    | new_prompt
    | llm
    | output_parser
)

chain.invoke({"question": "小明在哪里工作", "name": "主人"})

'小明在华为工作，是一个非常优秀的青年。所以，我可以告诉您，小明在华为工作，主人。'